# ADAS Object Detection - Training on Google Colab

This notebook trains YOLOv8 on BDD100K dataset using Google Colab's free GPU.

**Prerequisites:**
1. Upload `bdd100k_images_100k.zip` to Google Drive at `My Drive/datasets/`
2. Upload `bdd100k_labels_release.zip` to Google Drive at `My Drive/datasets/`

**Runtime**: Select GPU runtime (Runtime > Change runtime type > GPU)

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/Pranav1011/ADAS-OBJECT-DETECTION.git
%cd ADAS-OBJECT-DETECTION

In [ ]:
# Install dependencies
!pip install -q ultralytics opencv-python albumentations wandb pyyaml tqdm

## 2. Mount Google Drive & Copy Dataset

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create data directory
!mkdir -p data/raw data/processed/images/train data/processed/images/val data/processed/labels/train data/processed/labels/val

# Copy dataset from Drive
print("Copying images (this may take a few minutes)...")
!cp "/content/drive/MyDrive/datasets/bdd100k_images_100k.zip" data/raw/
print("Copying labels...")
!cp "/content/drive/MyDrive/datasets/bdd100k_labels_release.zip" data/raw/
print("Done!")

In [ ]:
# Extract dataset
print("Extracting images (this will take several minutes)...")
!cd data/raw && unzip -o -q bdd100k_images_100k.zip
print("Extracting labels...")
!cd data/raw && unzip -o -q bdd100k_labels_release.zip
print("Done!")

# Check extraction
!ls -la data/raw/

## 3. Convert BDD100K to YOLO Format

In [ ]:
import json
import shutil
from pathlib import Path
from tqdm import tqdm
import yaml
import random

# Configuration
CLASS_MAPPING = {
    "car": 0, "truck": 1, "bus": 2, "person": 3,
    "rider": 4, "bike": 5, "motor": 5,
    "traffic light": 6, "traffic sign": 7
}
CLASS_NAMES = ["car", "truck", "bus", "pedestrian", "cyclist", "motorcycle", "traffic_light", "traffic_sign"]
IMG_W, IMG_H = 1280, 720

MAX_TRAIN = 8000  # Use 8k for training
MAX_VAL = 2000    # Use 2k for validation

def convert_box(box):
    x1, y1, x2, y2 = box["x1"], box["y1"], box["x2"], box["y2"]
    cx = (x1 + x2) / 2 / IMG_W
    cy = (y1 + y2) / 2 / IMG_H
    w = (x2 - x1) / IMG_W
    h = (y2 - y1) / IMG_H
    return f"{cx:.6f} {cy:.6f} {w:.6f} {h:.6f}"

def process_split(img_dir, label_dir, out_img_dir, out_label_dir, max_images=None):
    out_img_dir.mkdir(parents=True, exist_ok=True)
    out_label_dir.mkdir(parents=True, exist_ok=True)
    
    images = list(img_dir.glob("*.jpg"))
    random.seed(42)
    random.shuffle(images)
    
    if max_images:
        images = images[:max_images * 2]  # Get more to account for no-label cases
    
    count = 0
    class_counts = {name: 0 for name in CLASS_NAMES}
    
    for img_path in tqdm(images, desc=f"Processing {img_dir.name}"):
        if max_images and count >= max_images:
            break
            
        json_path = label_dir / f"{img_path.stem}.json"
        if not json_path.exists():
            continue
        
        with open(json_path) as f:
            data = json.load(f)
        
        lines = []
        for frame in data.get("frames", []):
            for obj in frame.get("objects", []):
                cat = obj.get("category", "").lower()
                if cat not in CLASS_MAPPING:
                    continue
                if "box2d" not in obj:
                    continue
                class_id = CLASS_MAPPING[cat]
                box_str = convert_box(obj["box2d"])
                lines.append(f"{class_id} {box_str}")
                class_counts[CLASS_NAMES[class_id]] += 1
        
        if lines:
            shutil.copy(img_path, out_img_dir / img_path.name)
            with open(out_label_dir / f"{img_path.stem}.txt", "w") as f:
                f.write("\n".join(lines))
            count += 1
    
    return count, class_counts

# Process train and val
base = Path("data/raw")
out = Path("data/processed")

# Find the correct paths
if (base / "bdd100k/images/100k/train").exists():
    img_base = base / "bdd100k/images/100k"
    label_base = base / "bdd100k/labels/100k"
elif (base / "100k/train").exists():
    img_base = base / "100k"
    label_base = base / "100k"
else:
    raise FileNotFoundError("Could not find BDD100K data. Check extraction.")

print(f"Using images from: {img_base}")
print(f"Using labels from: {label_base}")
print()

train_count, train_classes = process_split(
    img_base / "train", label_base / "train",
    out / "images/train", out / "labels/train",
    max_images=MAX_TRAIN
)

val_count, val_classes = process_split(
    img_base / "val", label_base / "val",
    out / "images/val", out / "labels/val",
    max_images=MAX_VAL
)

print(f"\n{'='*50}")
print(f"Dataset Summary")
print(f"{'='*50}")
print(f"Train images: {train_count}")
print(f"Val images: {val_count}")
print(f"\nClass distribution (train):")
for name, count in sorted(train_classes.items(), key=lambda x: -x[1]):
    print(f"  {name}: {count:,}")

# Generate dataset.yaml
config = {
    "path": str(out.absolute()),
    "train": "images/train",
    "val": "images/val",
    "names": {i: n for i, n in enumerate(CLASS_NAMES)},
    "nc": len(CLASS_NAMES)
}
with open(out / "dataset.yaml", "w") as f:
    yaml.dump(config, f, sort_keys=False)
    
print(f"\nSaved dataset.yaml")
print(f"{'='*50}")

In [ ]:
# Verify dataset
!echo "Train images:" && ls data/processed/images/train | wc -l
!echo "Val images:" && ls data/processed/images/val | wc -l
!echo "\nDataset config:"
!cat data/processed/dataset.yaml

## 4. Train YOLOv8 Model

In [ ]:
from ultralytics import YOLO

# Load YOLOv8m model (medium - good balance of speed/accuracy)
model = YOLO('yolov8m.pt')

# Train
results = model.train(
    data='data/processed/dataset.yaml',
    epochs=100,
    batch=16,           # Adjust: 8 for T4, 16-32 for A100
    imgsz=640,
    patience=20,        # Early stopping
    device=0,
    project='runs/train',
    name='yolov8m-bdd100k',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    warmup_epochs=5,
    cos_lr=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
)

## 5. Evaluate Model

In [ ]:
# Evaluate on validation set
metrics = model.val(data='data/processed/dataset.yaml')

print("\n" + "="*50)
print("Evaluation Results")
print("="*50)
print(f"mAP@0.5: {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

## 6. Export Models

In [ ]:
# Create weights directory
!mkdir -p weights

# Copy best weights
!cp runs/train/yolov8m-bdd100k/weights/best.pt weights/best.pt

# Export to ONNX
print("Exporting to ONNX...")
model = YOLO('weights/best.pt')
model.export(format='onnx', imgsz=640, simplify=True, opset=12)
!mv weights/best.onnx weights/best.onnx

print("\nExported models:")
!ls -lh weights/

## 7. ONNX INT8 Quantization

In [ ]:
!pip install -q onnxruntime

In [ ]:
import numpy as np
import cv2
from pathlib import Path
import onnxruntime as ort
from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantFormat, QuantType

class CalibrationDataLoader(CalibrationDataReader):
    def __init__(self, calibration_images, input_name="images", img_size=640):
        self.image_paths = calibration_images
        self.input_name = input_name
        self.img_size = img_size
        self.index = 0

    def preprocess(self, image_path):
        img = cv2.imread(str(image_path))
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        img = img.transpose(2, 0, 1)
        return np.expand_dims(img, 0)

    def get_next(self):
        if self.index >= len(self.image_paths):
            return None
        image_path = self.image_paths[self.index]
        self.index += 1
        return {self.input_name: self.preprocess(image_path)}

    def rewind(self):
        self.index = 0

# Get calibration images
calibration_images = list(Path("data/processed/images/val").glob("*.jpg"))[:200]
print(f"Using {len(calibration_images)} images for calibration")

# Create calibration data reader
calibration_reader = CalibrationDataLoader(calibration_images)

# Quantize
print("Quantizing to INT8 (this may take a few minutes)...")
quantize_static(
    model_input="weights/best.onnx",
    model_output="weights/best_int8.onnx",
    calibration_data_reader=calibration_reader,
    quant_format=QuantFormat.QDQ,
    per_channel=True,
    activation_type=QuantType.QUInt8,
    weight_type=QuantType.QInt8
)

print("\nQuantized models:")
!ls -lh weights/*.onnx

## 8. TensorRT Export (FP16)

In [ ]:
# Export to TensorRT FP16
print("Exporting to TensorRT FP16...")
model = YOLO('weights/best.pt')
model.export(format='engine', imgsz=640, half=True, device=0)

!mv weights/best.engine weights/best_fp16.engine 2>/dev/null || true

print("\nAll exported models:")
!ls -lh weights/

## 9. Benchmark Models

In [ ]:
import time
import numpy as np

def benchmark_onnx(model_path, num_runs=100):
    session = ort.InferenceSession(model_path, providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
    input_name = session.get_inputs()[0].name
    dummy = np.random.randn(1, 3, 640, 640).astype(np.float32)
    
    # Warmup
    for _ in range(10):
        session.run(None, {input_name: dummy})
    
    # Benchmark
    times = []
    for _ in range(num_runs):
        start = time.perf_counter()
        session.run(None, {input_name: dummy})
        times.append((time.perf_counter() - start) * 1000)
    
    return np.mean(times), np.std(times)

def benchmark_pytorch(model_path, num_runs=100):
    import torch
    model = YOLO(model_path)
    dummy = torch.randn(1, 3, 640, 640).cuda()
    
    # Warmup
    for _ in range(10):
        model.model(dummy)
    
    # Benchmark
    times = []
    for _ in range(num_runs):
        torch.cuda.synchronize()
        start = time.perf_counter()
        model.model(dummy)
        torch.cuda.synchronize()
        times.append((time.perf_counter() - start) * 1000)
    
    return np.mean(times), np.std(times)

print("="*60)
print("BENCHMARK RESULTS")
print("="*60)
print(f"{'Model':<25} {'Size (MB)':<12} {'Latency (ms)':<15} {'FPS':<10}")
print("-"*60)

# PyTorch FP32
size = Path('weights/best.pt').stat().st_size / 1024 / 1024
mean, std = benchmark_pytorch('weights/best.pt')
print(f"{'PyTorch FP32':<25} {size:<12.1f} {mean:<15.2f} {1000/mean:<10.1f}")

# ONNX FP32
size = Path('weights/best.onnx').stat().st_size / 1024 / 1024
mean, std = benchmark_onnx('weights/best.onnx')
print(f"{'ONNX FP32':<25} {size:<12.1f} {mean:<15.2f} {1000/mean:<10.1f}")

# ONNX INT8
if Path('weights/best_int8.onnx').exists():
    size = Path('weights/best_int8.onnx').stat().st_size / 1024 / 1024
    mean, std = benchmark_onnx('weights/best_int8.onnx')
    print(f"{'ONNX INT8':<25} {size:<12.1f} {mean:<15.2f} {1000/mean:<10.1f}")

print("="*60)

## 10. Test Inference

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Load model
model = YOLO('weights/best.pt')

# Get sample image
sample_images = list(Path('data/processed/images/val').glob('*.jpg'))[:3]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, img_path in zip(axes, sample_images):
    results = model(str(img_path), verbose=False)[0]
    annotated = results.plot()
    ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    ax.axis('off')
    ax.set_title(f"{len(results.boxes)} detections")

plt.tight_layout()
plt.savefig('sample_detections.png', dpi=150, bbox_inches='tight')
plt.show()

print("Saved sample_detections.png")

## 11. Save to Google Drive

In [ ]:
# Create output directory in Drive
!mkdir -p "/content/drive/MyDrive/adas-detection-results"

# Copy weights
!cp -r weights "/content/drive/MyDrive/adas-detection-results/"

# Copy training results
!cp -r runs/train/yolov8m-bdd100k "/content/drive/MyDrive/adas-detection-results/"

# Copy sample detections
!cp sample_detections.png "/content/drive/MyDrive/adas-detection-results/"

print("\nSaved to Google Drive!")
print("Location: My Drive/adas-detection-results/")
!ls -lh "/content/drive/MyDrive/adas-detection-results/weights/"

## 12. Summary

Training complete! Your trained models are saved to:
- `My Drive/adas-detection-results/weights/best.pt` - PyTorch model
- `My Drive/adas-detection-results/weights/best.onnx` - ONNX FP32
- `My Drive/adas-detection-results/weights/best_int8.onnx` - ONNX INT8
- `My Drive/adas-detection-results/weights/best_fp16.engine` - TensorRT FP16

Next steps:
1. Download the weights to your local machine
2. Place them in `weights/` folder of your project
3. Run the Gradio demo or FastAPI server